In [12]:
import pandas as pd
import numpy as np
from numpy import nan
import math
import matplotlib.pyplot as plt
%matplotlib inline
import random
import seaborn as sns

In [150]:
merged_image = pd.read_csv('./data/merged_image_match_facial_recognition.csv')
matched = merged_image[merged_image['image_match']>0.4]
merged_df = pd.read_csv('./data/merged.csv')
merged_df['bd'] = merged_df['bd'].astype(str).str[:4] #just taking the year
merged_df.columns.values
merged_df
#looking at yelp and tinder merged dataframe after cross matching

,Unnamed: 0,review_id,review_url,text,time_created,user_id_x,user_profile_url,user_image_url,user_name,user_first_name,...,bio,bd,name,gender,city,distance,company,job_title,school,first_photo_url
0,0,LYzz3WgMHYiauk2S1S_vNA,https://www.yelp.com/biz/laso-american-canyon?...,Meant to have completed this review a few mont...,2023-03-29 06:26:46,Ng6C_umIMragw2SvfKjPwQ,https://www.yelp.com/user_details?userid=Ng6C_...,https://s3-media4.fl.yelpcdn.com/photo/4vJfZsT...,Robert N.,Robert,...,You miss 100% of the shots you don’t take\n-Wa...,1999,Robert,male,San Francisco,1,NaN,MA,San Francisco State University,https://images-ssl.gotinder.com/u/m2uXqMpdgCKr...
1,1,_zG2q3TZiBSQtaNZhtrqkw,https://www.yelp.com/biz/italy-on-gilman-berke...,I really love this place. I've been several ti...,2023-04-02 14:28:12,p2wuTYAzm-gedYKHOuSEZg,https://www.yelp.com/user_details?userid=p2wuT...,https://s3-media4.fl.yelpcdn.com/photo/yefuIjr...,Robert L.,Robert,...,You miss 100% of the shots you don’t take\n-Wa...,1999,Robert,male,San Francisco,1,NaN,MA,San Francisco State University,https://images-ssl.gotinder.com/u/m2uXqMpdgCKr...
2,2,Na37yEmuh637dsj9ZaX8Ew,https://www.yelp.com/biz/ramen-seas-sunnyvale?...,Would you spend $52.76 for two bowls of ramen?...,2023-01-30 20:52:25,yZ-vbhFPnhInlnxxEQyVdw,https://www.yelp.com/user_details?userid=yZ-vb...,https://s3-media1.fl.yelpcdn.com/photo/BC0epdZ...,Robert C.,Robert,...,You miss 100% of the shots you don’t take\n-Wa...,1999,Robert,male,San Francisco,1,NaN,MA,San Francisco State University,https://images-ssl.gotinder.com/u/m2uXqMpdgCKr...
3,3,7ENTwwqlqrzJJOqWrsQi0A,https://www.yelp.com/biz/the-savory-grain-rich...,Part of our party arrived about an hour before...,2023-01-08 06:00:38,ttbp3nlBaQpxhP3JRTCviQ,https://www.yelp.com/user_details?userid=ttbp3...,https://s3-media4.fl.yelpcdn.com/photo/H1DlOvn...,Robert C.,Robert,...,You miss 100% of the shots you don’t take\n-Wa...,1999,Robert,male,San Francisco,1,NaN,MA,San Francisco State University,https://images-ssl.gotinder.com/u/m2uXqMpdgCKr...
4,4,uIdJ4wleIk-V8fiz2qSizQ,https://www.yelp.com/biz/mendocino-farms-san-r...,I love the quality and freshness of all the fo...,2023-03-07 14:53:14,QYUZD1Tn0UzrTbUG08lpiw,https://www.yelp.com/user_details?userid=QYUZD...,https://s3-media1.fl.yelpcdn.com/photo/isCBYiB...,Robert M.,Robert,...,You miss 100% of the shots you don’t take\n-Wa...,1999,Robert,male,San Francisco,1,NaN,MA,San Francisco State University,https://images-ssl.gotinder.com/u/m2uXqMpdgCKr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14883,14883,Yk4NWFVmJXQ1MTITb-JPfg,https://www.yelp.com/biz/the-keg-steakhouse-ba...,"Perfect Place to eat good food, take a drink a...",2023-04-15 13:10:27,3ATZL71xHxfjpi2CRZx2ww,https://www.yelp.com/user_details?userid=3ATZL...,https://s3-media4.fl.yelpcdn.com/photo/55LK4fB...,Alejandro H.,Alejandro,...,NaN,1993,Alejandro,not displayed,San Pablo,1,NaN,NaN,Research and Advanced Studies Centre of the IPN,https://images-ssl.gotinder.com/u/vd4tCCMHDKgp...
14884,14884,xWV9tUC4izLNWZgk7Vb5CQ,https://www.yelp.com/biz/stem-japanese-eatery-...,The first time coming here was many years ago ...,2022-11-14 10:18:22,3EQ0orqLPmXLaU8G18UOAA,https://www.yelp.com/user_details?userid=3EQ0o...,https://s3-media2.fl.yelpcdn.com/photo/_RVI7rL...,JT C.,JT,...,YHVH\n6ft\nAmbivert\nINFJ-A\nWorking out\nTrav...,1989,JT,not displayed,NaN,25,NaN,Owner,"California State University, East Bay",https://images-ssl.gotinder.com/u/bLUW1z7FVjdX...
14885,14885,xWV9tUC4izLNWZgk7Vb5CQ,https://www.yelp.com/biz/stem-japanese-eatery-...,The first time coming here was many years ago ...,2022-11-14 10:18:22,3EQ0orqLPmXLaU8G18UOAA,https://www.yelp.com/user_details?userid=3EQ0o...,https://s3-media2.fl.yelpcdn.com/photo/_RVI7rL...,JT C.,JT,...,heart as big as my schnoz,1995,JT,not displayed,El Cerrito,7,NaN,NaN,"University of California, Berkeley",https://images-ssl.gotinder.com/u/qXSdCkzCrjcE...
14886,14886,GkVM84wazcWizVJbJB

These are the 8 identities we will be checking

In [31]:
matched = matched.rename(columns={"4": "gender", "5": "city", "6": "distance", "7": "company", "8": "job_title", "9": "school"})
matched

,Unnamed: 0,review_id,review_url,text,time_created,user_id,user_profile_url,user_image_url,user_name,user_first_name,...,3,gender,city,distance,company,job_title,school,10,facial_recognition,image_match
190,190,99jYWp5PxQrYUFWKi23MHw,https://www.yelp.com/biz/longboards-restaurant...,"Longboards Restaurant & Bar\n 72 Wharf St, Sal...",2023-01-26 17:42:08,b_Mcyk2P5DkSz_fx0Zu6Ag,https://www.yelp.com/user_details?userid=b_Mcy...,https://s3-media2.fl.yelpcdn.com/photo/qq9ocyS...,Ryan M.,Ryan,...,Ryan,not displayed,Mountain View,10000,NaN,RN,NaN,https://images-ssl.gotinder.com/u/nmXngUZtv2B6...,False,0.482801
203,203,wBLywAyohpbBsdRRFDE_Lw,https://www.yelp.com/biz/northeast-chinese-res...,First time visiting this place. We moved from ...,2023-02-03 21:18:41,vetI1F4btXdQAe5G2A2ciA,https://www.yelp.com/user_details?userid=vetI1...,https://s3-media3.fl.yelpcdn.com/photo/hyKqGkO...,Ryan H.,Ryan,...,Ryan,not displayed,Mountain View,10000,NaN,RN,NaN,https://images-ssl.gotinder.com/u/nmXngUZtv2B6...,NaN,0.553566
212,212,q4cbO40ZvH40xPEcJMpIgw,https://www.yelp.com/biz/la-carreta-mexican-re...,This restaurant has come along way in its shor...,2019-06-05 04:42:16,7p4z1bXHZ4_Y9mIa-5f5yA,https://www.yelp.com/user_details?userid=7p4z1...,https://s3-media1.fl.yelpcdn.com/photo/7kSMaiX...,Ryan G.,Ryan,...,Ryan,not displayed,Mountain View,10000,NaN,RN,NaN,https://images-ssl.gotinder.com/u/nmXngUZtv2B6...,NaN,0.474308
236,236,U3XT02fpDh3EX5eJ-WIX-g,https://www.yelp.com/biz/fish-and-bird-sousaku...,Service was regular.\nAmbience is mid went in ...,2023-04-03 15:54:20,YQY4JYV1hOy5ogKXt82DPQ,https://www.yelp.com/user_details?userid=YQY4J...,https://s3-media2.fl.yelpcdn.com/photo/ihqZXGW...,Daniel D.,Daniel,...,Daniel,not displayed,Berkeley,1,NaN,Assistant Director,"University of California, Berkeley",https://images-ssl.gotinder.com/u/iBA2B2tnneMA...,NaN,0.401428
242,242,MLojmH0n_Cqiu7PmLNfYXg,https://www.yelp.com/biz/sushi-yoshizumi-san-m...,Visited after it raises its price to $325 per ...,2022-12-28 15:31:06,leftGY0EXgcotnOlRXYhzw,https://www.yelp.com/user_details?userid=leftG...,https://s3-media4.fl.yelpcdn.com/photo/9eb2RLP...,Daniel X.,Daniel,...,Daniel,not displayed,Berkeley,1,NaN,Assistant Director,"University of California, Berkeley",https://images-ssl.gotinder.com/u/iBA2B2tnneMA...,NaN,0.410927
335,335,j0YUs1s6AsvfKuqfBcgxSg,https://www.yelp.com/biz/tayyibaat-meat-and-gr...,I came across this place due to a craving for ...,2023-02-08 21:37:51,kONWoFdMPoBpJdjfKeV_2w,https://www.yelp.com/user_details?userid=kONWo...,https://s3-media3.fl.yelpcdn.com/photo/tbYs7W1...,Matthew T.,Matthew,...,Matthew,male,NaN,1,NaN,NaN,"University of California, Berkeley",https://images-ssl.gotinder.com/u/tC8Wby2VwTBu...,NaN,0.433841
350,350,S2pQetv4DJefhDZzYa0Bhg,https://www.yelp.com/biz/q-restaurant-boston?a...,"Q's unassuming staff and environment, it's del...",2023-02-26 15:59:11,NXmnJeYIHd-m8L7kFdPcUA,https://www.yelp.com/user_details?userid=NXmnJ...,https://s3-media2.fl.yelpcdn.com/photo/XZGLvR7...,Matthew M.,Matthew,...,Matthew,male,NaN,1,NaN,NaN,"University of California, Berkeley",https://images-ssl.gotinder.com/u/tC8Wby2VwTBu...,False,0.521787
376,376,u3hjHRP-juR1FQLS2GXxig,https://www.yelp.com/biz/sunflower-caffe-sonom...,Granola Waffle \nWaffle is light airy.. not d...,2023-04-27 19:16:51,R7vRcpDhiiYMgzlJ-I7bKA,https://www.yelp.com/user_details?userid=R7vRc...,https://s3-media4.fl.yelpcdn.com/photo/ZGceuns...,Levi S.,Levi,...,Levi,not displayed,NaN,9,NaN,NaN,San Francisco State University,https://images-ssl.gotinder.com/u/3XMBzLhsBZAR...,NaN,0.453324


Wanted to look into l-diversity based on our merged table, which is the table we are working with, which is a combination of 2 public tables to see how secure the table is after combining all the information based on what we know. I am looking at user_name as it is one of the more specific quasi-identifiers that are used and since names are a very helpful clue for connecting social media platforms.

In [144]:
merged_df["user_name"].value_counts()

Kevin L.      240
Chris M.      221
Michael U.    168
Chris L.      136
John D.       135
             ... 
Patrick B.      1
Shannon Z.      1
Shannon B.      1
Shannon D.      1
Elena C.        1
Name: user_name, Length: 2159, dtype: int64

In [145]:
name = merged_df['name'].value_counts()
single = name[name==1]
single

Lorena        1
Kevan         1
Brooks        1
Arun          1
Buck          1
             ..
Maximilian    1
Kegan         1
Cherry        1
Mady          1
Rin           1
Name: name, Length: 69, dtype: int64

As you can see here, there are a lot of Kevins, Chris', Michaels, etc. but there are still a lot of very specific names like Elena C. and Shannon D.. With these, this means that the equivalence classes for names leads to a k-anonymity score of only 1 for the whole table since that is the only number that satisfies all of the equivalence classes.This makes it very easy for anyone to point out who these people are in the dataset and know the urls to their Yelp profile, which can then reveal a lot of information about their lifestyle and the places they frequent.

I also looked at different combinations of equivalence classes, and the minimum k was still 1 for many. This means that there are no other people that could mask their identitity, and if someone looks up the name Rin and Kegan, they will be identified and and known within the dataset.

In [211]:
equivalence_classes = merged_df[{"bd", "gender", "user_first_name"}].drop_duplicates()
k_list = merged_df.groupby(["bd", "gender", "user_first_name"], dropna = False).size().reset_index(name='k')
ec_sorted = k_list.sort_values(["bd", "gender", "user_first_name"], ascending = [True, True, True])
k = min(ec_sorted["k"])
l_values = merged_df[{"bd", "gender", "user_first_name"}].value_counts()
p = l_values/14888
entropy = 10**((-sum(l_values * np.log10(p)))/14888)
entropy 

335.43652994190336

Satisfies 723 Distinct l-diversity and is 335.4365 entropy diverse

In [212]:
#trying it out with less quasi-identifiers, this time with just birthday year and city
equivalence_classes = merged_df[{"bd", "city"}].drop_duplicates()
k_list = merged_df.groupby(["bd", "city"], dropna = False).size().reset_index(name='k')
ec_sorted = k_list.sort_values(["bd", "city"], ascending = [True, True])
k = min(ec_sorted["k"])
l_values = merged_df[{"bd", "city"}].value_counts()
p = l_values/14888
entropy = 10**((-sum(l_values * np.log10(p)))/14888)
entropy 

17.217792528458517

Satisfies 190 Distinct l-diversity and 17.2178 entropy diverse

In [210]:
#trying it out with less quasi-identifiers, this time with just birthday year
equivalence_classes = merged_df[{"bd"}].drop_duplicates()
k_list = merged_df.groupby(["bd"], dropna = False).size().reset_index(name='k')
ec_sorted = k_list.sort_values(["bd"], ascending = True)
k = min(ec_sorted["k"])
len(equivalence_classes)
l_values = merged_df[{"bd"}].value_counts()
p = l_values/14888
entropy = 10**((-sum(l_values * np.log10(p)))/14888)
entropy 

13.508350698796862

Satisfies 21 Distinct l-diversity  and 13.5084 entropy diverse

I noticed that for the first two combinations of quasi-identifiers that the smallest k was 1, meaning that there was at least one unique entry for those combinations. To explore further, I wanted to look to see how many unique names we had that only had a single entry.

Now looking into l-diveristy and t-closeness, we wanted to explore delta-presence to 

In [213]:
yelp_identities = pd.read_csv('./data/reviews_with_first_name.csv')
tinder_identities = pd.read_csv('./data/tinder_users_no_dup.csv')

In [41]:
quasi_identifiers = {"gender", "city", "school"}
identities = matched[{"gender", "city", "school", "user_name"}]
identities

,gender,city,school,user_name
190,not displayed,Mountain View,NaN,Ryan M.
203,not displayed,Mountain View,NaN,Ryan H.
212,not displayed,Mountain View,NaN,Ryan G.
236,not displayed,Berkeley,"University of California, Berkeley",Daniel D.
242,not displayed,Berkeley,"University of California, Berkeley",Daniel X.
335,male,NaN,"University of California, Berkeley",Matthew T.
350,male,NaN,"University of California, Berkeley",Matthew M.
376,not displayed,NaN,San Francisco State University,Levi S.


Looking at this, we can see that just looking at the 8 different identities that we were able to successfully match based on multiple factors such as city and facial expression. We can see that if we just take into accoun thte quasi-identifiers that only Levi is the only person that doesn't have the same exact quasi-identifiers as another person. However, this is a small list anyways, but considering this, there are a lot of overlap for the other individuals, which can help mask them more in terms of l-diversity.

In [121]:
iden = matched[{"gender", "city", "school"}].drop_duplicates()
iden

,gender,city,school
190,not displayed,Mountain View,NaN
236,not displayed,Berkeley,"University of California, Berkeley"
335,male,NaN,"University of California, Berkeley"
376,not displayed,NaN,San Francisco State University


In [130]:
pqID = merged_df[{ "gender", "city", "school"}]
iqID = iden.iloc[0, :]
left, right = pqID.align(iqID, axis=1, copy=False)
check = left == right
check[(check["city"] == True) & (check["gender"] == True) & (check["school"] == True)]
prob_of_identifying = 0

Series([], dtype: int64)

In [148]:
pqID = merged_df[{ "gender", "city", "school"}]
iqID = iden.iloc[1, :]
left, right = pqID.align(iqID, axis=1, copy=False)
check = left == right
check[(check["city"] == True) & (check["gender"] == True) & (check["school"] == True)]
#prob_of_identifying = 1/532

,gender,city,school
1544,True,True,True
1548,True,True,True
1560,True,True,True
1564,True,True,True
1576,True,True,True
...,...,...,...
14658,True,True,True
14660,True,True,True
14662,True,True,True
14664,True,True,True


In [132]:
pqID = merged_df[{ "gender", "city", "school"}]
iqID = iden.iloc[2, :]
left, right = pqID.align(iqID, axis=1, copy=False)
check = left == right
check[(check["city"] == True) & (check["gender"] == True) & (check["school"] == True)]
prob_of_identifying = 0

In [133]:
pqID = merged_df[{ "gender", "city", "school"}]
iqID = iden.iloc[3, :]
left, right = pqID.align(iqID, axis=1, copy=False)
check = left == right
check[(check["city"] == True) & (check["gender"] == True) & (check["school"] == True)]
prob_of_identifying = 0

Extra just to see what the numbers looked like in terms of the difference between positive reviews and if there that could arise from privacy concerns

In [266]:
def words_in_texts(words, texts):
    indicator_array = []
    for x in texts:
        boo = []
        for i in words:
            if i in x:
                boo.append(1)
            else:
                boo.append(0)        
        indicator_array.append(boo)     
    return np.array(indicator_array)

In [280]:
l = ['amazing', 'delicious', 'worth', 'satisfied', 'awful', 'negative','concern', 'angry', 'rude']
ba = words_in_texts(l, merged_df['text'])
words = pd.DataFrame({"words": l, "count": sum(ba)})
words

array([710, 870, 206,  22,  19,  40,  39,  17,  49])